<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import threading
import shutil
from datetime import datetime
today = datetime.now().strftime("%m%d")

# Google Colab 환경 확인
try:
    from google.colab import files
    IN_COLAB = True
    # 위젯 설치 및 임포트
    try:
        import ipywidgets as widgets
        from IPython.display import display
        # clear_output을 별도로 import
        try:
            from IPython.display import clear_output as ipy_clear_output
        except:
            ipy_clear_output = None
        WIDGETS_AVAILABLE = True
    except ImportError:
        try:
            print("🔧 위젯 라이브러리 설치 중...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])
            import ipywidgets as widgets
            from IPython.display import display
            try:
                from IPython.display import clear_output as ipy_clear_output
            except:
                ipy_clear_output = None
            WIDGETS_AVAILABLE = True
        except:
            print("⚠️ 위젯 설치 실패, 기본 모드로 실행됩니다.")
            widgets = None
            ipy_clear_output = None
            WIDGETS_AVAILABLE = False
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중입니다.")
    widgets = None
    ipy_clear_output = None
    WIDGETS_AVAILABLE = False

def install_ytdlp():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "yt-dlp"])
    except subprocess.CalledProcessError:
        print("❌ yt-dlp 설치에 실패했습니다.")
        return False
    return True

def clear_existing_folder(folder_name):
    if os.path.exists(folder_name):
        try:
            for filename in os.listdir(folder_name):
                file_path = os.path.join(folder_name, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            print(f"🧹 기존 폴더 '{folder_name}' 내용을 모두 삭제했습니다.")
            return True
        except Exception as e:
            print(f"❌ 폴더 정리 중 오류 발생: {str(e)}")
            return False
    return True

def get_video_title(url):
    """유튜브 URL에서 영상 제목만 가져오기"""
    try:
        result = subprocess.run(
            ['yt-dlp', '--get-title', url],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return None
    except Exception:
        return None

def create_download_folder():
    """다운로드 폴더 생성"""
    folder_name = "영상저장함"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"📁 폴더 생성: {folder_name}")
    else:
        print(f"📁 기존 폴더 사용: {folder_name}")

    return folder_name

def create_zip_file(folder_name):
    """폴더를 ZIP으로만 압축 (자동 다운로드 없음)"""
    zip_filename = f"{today}_유튜브다운로드.zip"
    if os.path.exists(zip_filename):
        os.remove(zip_filename)

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        for file in os.listdir(folder_name):
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, file)
    print(f"📦 ZIP 파일 생성 완료: {zip_filename}")
    return zip_filename

class YouTubeDownloader:
    def __init__(self):
        self.urls = []
        self.quality = 'normal'
        self.folder_name = None

    def create_widgets(self):
        """위젯 UI 생성"""
        if not IN_COLAB or not WIDGETS_AVAILABLE:
            return self.get_video_urls_legacy()

        # 공통 스타일 설정
        header_style = {'description_width': '0px'}
        input_style = {'description_width': '140px'}
        button_layout = widgets.Layout(height='35px', margin='5px')

        # 🎨 헤더 섹션
        title = widgets.HTML(
            value="""
            <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        border-radius: 15px; margin-bottom: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.1);'>
                <h1 style='color: white; margin: 0; font-family: Arial, sans-serif; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);'>
                    🎥 YouTube Downloader
                </h1>
                <p style='color: rgba(255,255,255,0.9); margin: 5px 0 0 0; font-size: 14px;'>
                    빠르고 간편한 유튜브 동영상 다운로더
                </p>
            </div>
            """,
            layout=widgets.Layout(margin='0 0 25px 0')
        )

        # 📝 URL 입력 섹션
        url_section_title = widgets.HTML(
            value="<h3 style='margin: 0 0 15px 0; color: #333; font-weight: bold;'>🔗 링크 입력</h3>"
        )

        self.url_input = widgets.Text(
            placeholder='https://www.youtube.com/watch?v=... 또는 https://youtu.be/...',
            description='YouTube URL:',
            style=input_style,
            layout=widgets.Layout(width='100%', height='40px')
        )

        # 버튼 그룹
        self.add_button = widgets.Button(
            description='📋 링크 추가',
            button_style='primary',
            tooltip='현재 입력된 링크를 목록에 추가합니다',
            layout=widgets.Layout(width='140px', **button_layout)
        )

        self.clear_button = widgets.Button(
            description='🗑️ 목록 초기화',
            button_style='warning',
            tooltip='추가된 모든 링크를 삭제합니다',
            layout=widgets.Layout(width='140px', **button_layout)
        )

        url_buttons = widgets.HBox(
            [self.add_button, self.clear_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0 0 0')
        )

        # 🎯 화질 선택 섹션
        quality_section_title = widgets.HTML(
            value="<h3 style='margin: 25px 0 15px 0; color: #333; font-weight: bold;'>🎯 화질 설정</h3>"
        )

        self.quality_radio = widgets.RadioButtons(
            options=[
                ('📱 일반화질 (720p 이하) - 빠른 다운로드, 작은 용량', 'normal'),
                ('🎬 고화질 (1080p 이상) - 최고 품질, 큰 용량', 'high')
            ],
            value='normal',
            layout=widgets.Layout(margin='10px 0 0 20px')
        )

        # 🚀 다운로드 섹션
        download_section_title = widgets.HTML(
            value="<h3 style='margin: 25px 0 15px 0; color: #333; font-weight: bold;'>🚀 다운로드</h3>"
        )

        self.download_button = widgets.Button(
            description='⬇️ 다운로드 시작',
            button_style='success',
            tooltip='선택된 모든 영상을 다운로드합니다',
            layout=widgets.Layout(
                width='200px',
                height='45px',
                margin='10px 0 0 0',
                border='2px solid #28a745'
            )
        )

        download_section = widgets.HBox(
            [self.download_button],
            layout=widgets.Layout(justify_content='center')
        )

        # 📋 출력 영역
        output_title = widgets.HTML(
            value="<h3 style='margin: 30px 0 15px 0; color: #333; font-weight: bold;'>📋 작업 로그</h3>"
        )

        self.output = widgets.Output(
            layout=widgets.Layout(
                border='2px solid #e9ecef',
                border_radius='10px',
                padding='15px',
                max_height='400px',
                overflow='auto'
            )
        )

        # 이벤트 연결
        self.add_button.on_click(self.add_url)
        self.clear_button.on_click(self.clear_urls)
        self.download_button.on_click(self.start_download)
        self.url_input.on_submit(self.add_url_on_enter)

        # 📱 전체 레이아웃 구성
        main_container = widgets.VBox([
            title,

            # URL 입력 섹션
            url_section_title,
            self.url_input,
            url_buttons,

            # 화질 선택 섹션
            quality_section_title,
            self.quality_radio,

            # 다운로드 섹션
            download_section_title,
            download_section,

            # 출력 섹션
            output_title,
            self.output
        ], layout=widgets.Layout(
            padding='20px',
            border='1px solid #dee2e6',
            border_radius='15px',
            background_color='#f8f9fa',
            box_shadow='0 8px 25px rgba(0,0,0,0.1)'
        ))

        # 초기 메시지 표시
        with self.output:
            self._show_initial_message()

        display(main_container)

    def add_url_on_enter(self, text_widget):
        """엔터키로 URL 추가"""
        self.add_url(None)

    def add_url(self, button):
        """URL 추가"""
        url = self.url_input.value.strip()

        with self.output:
            if not url:
                print("❌ URL을 입력해주세요!")
                return

            if 'youtube.com' not in url and 'youtu.be' not in url:
                print("❌ 올바른 유튜브 링크를 입력해주세요!")
                return

            if url in self.urls:
                print("⚠️  이미 추가된 링크입니다!")
                return

            self.urls.append(url)
            print(f"✅ [{len(self.urls)}] 링크가 추가되었습니다")

            # 제목 확인 (백그라운드에서)
            print(f"🔄 [{len(self.urls)}] 영상 정보 확인 중...")
            threading.Thread(target=self.check_title, args=(url, len(self.urls))).start()

        self.url_input.value = ''  # 입력창 초기화

    def check_title(self, url, index):
        """백그라운드에서 제목 확인"""
        title = get_video_title(url)
        with self.output:
            if title:
                print(f"🎵 [{index}] 📹 {title}")
            else:
                print(f"⚠️  [{index}] 제목 확인 실패 (링크는 정상 추가됨)")

    def clear_urls(self, button):
        """URL 목록 초기화"""
        if not self.urls:
            with self.output:
                print("💡 삭제할 링크가 없습니다.")
                return

        count = len(self.urls)
        self.urls = []

        # 출력 영역을 안전하게 초기화
        try:
            with self.output:
                # 여러 방법으로 화면 정리 시도
                if ipy_clear_output and callable(ipy_clear_output):
                    ipy_clear_output(wait=True)
                elif hasattr(self.output, 'clear_output') and callable(self.output.clear_output):
                    self.output.clear_output(wait=True)
                else:
                    # 대안: 여러 줄 띄우기로 시각적 정리
                    print("\n" * 10)

                self._show_initial_message()
                print(f"🧹 {count}개의 링크가 모두 삭제되었습니다.")
        except Exception as e:
            # 최후의 수단: 단순 메시지만 표시
            with self.output:
                print(f"🧹 {count}개의 링크가 모두 삭제되었습니다.")
                print("✨ 새로운 링크를 추가해주세요!")

    def _show_initial_message(self):
        """초기 안내 메시지 표시"""
        print("✨ YouTube Downloader가 준비되었습니다!")
        print("📌 사용법:")
        print("   1️⃣ 위 입력창에 YouTube 링크를 붙여넣기")
        print("   2️⃣ '링크 추가' 버튼 클릭 또는 Enter 키")
        print("   3️⃣ 원하는 만큼 링크 추가 반복")
        print("   4️⃣ 화질 설정 선택")
        print("   5️⃣ '다운로드 시작' 버튼 클릭")
        print("\n" + "="*50)

    def start_download(self, button):
        """다운로드 시작"""
        with self.output:
            if not self.urls:
                print("❌ 다운로드할 링크를 먼저 추가해주세요!")
                return

            self.quality = self.quality_radio.value
            self.folder_name = create_download_folder()

            print(f"\n{'='*60}")
            print(f"🚀 다운로드 시작! (총 {len(self.urls)}개 영상)")
            quality_text = "고화질 (1080p+)" if self.quality == 'high' else "일반화질 (720p)"
            print(f"📺 선택된 화질: {quality_text}")
            print(f"📁 저장 위치: {self.folder_name}")
            print("="*60)

            has_downloads = self.download_videos()

            if has_downloads and IN_COLAB:
                print("\n" + "="*60)
                print("📦 ZIP 파일 생성 및 다운로드 준비")
                print("="*60)
                zip_filename = create_zip_file(self.folder_name)
                print("⬇️ 파일 다운로드를 시작합니다...")
                files.download(zip_filename)
                clear_existing_folder(self.folder_name)
                print("✅ 다운로드 완료! 브라우저에서 파일을 확인하세요.")
            elif has_downloads:
                print("💻 로컬 환경: 파일들이 폴더에 저장되었습니다.")

    def download_videos(self):
        """비디오 다운로드"""
        # 화질 설정
        if self.quality == 'high':
            format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
            print("🎬 고화질 모드로 다운로드합니다.")
        else:
            format_option = "best[height<=720]/best"
            print("📱 일반화질 모드로 다운로드합니다.")

        success_count = 0
        failed_urls = []

        for i, url in enumerate(self.urls, 1):
            print(f"\n📥 [{i}/{len(self.urls)}] 다운로드 진행 중...")

            try:
                cmd = [
                    "yt-dlp",
                    "-f", format_option,
                    "-o", f"{self.folder_name}/%(title)s.%(ext)s",
                    "--no-warnings",
                    url
                ]
                result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

                if result.returncode == 0:
                    print(f"✅ [{i}] 다운로드 성공!")
                    success_count += 1
                else:
                    print(f"❌ [{i}] 다운로드 실패!")
                    error_msg = result.stderr.strip()
                    if error_msg:
                        print(f"   💬 오류 원인: {error_msg[:100]}")
                    failed_urls.append(url)

            except Exception as e:
                print(f"❌ [{i}] 처리 중 예외 발생: {str(e)[:100]}")
                failed_urls.append(url)

        # 결과 요약
        print("\n" + "="*60)
        print("📊 다운로드 결과 요약")
        print("="*60)
        print(f"✅ 성공: {success_count}개 영상")
        print(f"❌ 실패: {len(failed_urls)}개 영상")

        if success_count > 0:
            print(f"📁 저장 위치: {os.path.abspath(self.folder_name)}")

        if failed_urls:
            print(f"\n❌ 실패한 링크 목록:")
            for idx, url in enumerate(failed_urls, 1):
                print(f"   {idx}. {url[:60]}...")

        return success_count > 0

    def get_video_urls_legacy(self):
        """로컬 환경용 레거시 입력 방식"""
        urls = []
        print("\n" + "="*80)
        print("🔗 유튜브 링크 입력")
        print("="*80)
        print("1️⃣ 다운로드할 유튜브 링크를 입력해주세요.")
        print("2️⃣ 링크를 하나씩 입력하고 엔터를 누르세요.")
        print("3️⃣ 모든 링크를 입력했으면 '다운로드'를 입력하세요.")
        print("-"*80)

        while True:
            try:
                print(f"\n🔗 링크 #{len(urls)+1} 입력:")
                url_input = input("   │ ").strip()
                if url_input.lower() == '다운로드':
                    if urls:
                        print(f"\n✅ 총 {len(urls)}개 링크가 입력되었습니다.")
                        break
                    else:
                        print("❌ 최소 하나의 링크를 입력해주세요!")
                        continue
                if url_input:
                    if 'youtube.com' in url_input or 'youtu.be' in url_input:
                        urls.append(url_input)
                        print(f"✅ 추가됨: {len(urls)}번째 영상")
                    else:
                        print("❌ 올바른 유튜브 링크를 입력해주세요!")
                else:
                    print("💡 링크를 입력하거나 '다운로드'를 입력해주세요.")
            except KeyboardInterrupt:
                print("\n\n⏹️  입력이 중단되었습니다.")
                return []

        # 제목 확인은 입력이 끝난 뒤
        print("\n🎥 영상 확인 중...")
        for i, url in enumerate(urls, 1):
            title = get_video_title(url)
            if title:
                print(f"[{i}] {title}")
            else:
                print(f"[{i}] 제목 확인 실패")
        return urls

def get_quality_choice_legacy():
    """로컬 환경용 화질 선택"""
    print("\n" + "="*40)
    print("🎯 화질 선택")
    print("="*40)
    print("1. 📱 일반화질 (720p 이하, 빠른 다운로드)")
    print("2. 🎬 고화질 (최고 품질, 용량 큼)")
    print("-"*40)

    while True:
        choice = input("선택하세요 (1 또는 2): ").strip()
        if choice == '1':
            print("✅ 일반화질 모드가 선택되었습니다.")
            return 'normal'
        elif choice == '2':
            print("✅ 고화질 모드가 선택되었습니다.")
            return 'high'
        else:
            print("❌ 1 또는 2를 입력해주세요!")

def main():
    """메인 함수"""
    try:
        subprocess.run(['yt-dlp', '--version'], check=True, capture_output=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        if not install_ytdlp():
            return

    try:
        downloader = YouTubeDownloader()

        if IN_COLAB and widgets:
            # Colab 환경에서 위젯 사용
            print("🎨 위젯 UI를 사용합니다. 스크롤 문제가 해결됩니다!")
            downloader.create_widgets()
        else:
            # 로컬 환경에서 기존 방식 사용
            urls = downloader.get_video_urls_legacy()
            if not urls:
                return

            quality = get_quality_choice_legacy()
            folder_name = create_download_folder()

            downloader.urls = urls
            downloader.quality = quality
            downloader.folder_name = folder_name

            has_downloads = downloader.download_videos()

            if has_downloads and IN_COLAB:
                print("\n" + "="*50)
                print("📦 압축 및 다운로드 준비")
                print("="*50)
                zip_filename = create_zip_file(folder_name)
                files.download(zip_filename)
                clear_existing_folder(folder_name)
                print("⬇️ ZIP 파일 자동 다운로드 시작")
            elif has_downloads:
                print("💻 로컬 환경: 파일들이 폴더에 저장되었습니다.")

            input("\n아무 키나 눌러서 종료하세요...")

    except KeyboardInterrupt:
        print("\n\n⏹️  사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 오류가 발생했습니다: {str(e)}")
        if not IN_COLAB:
            input("\n아무 키나 눌러서 종료하세요...")

if __name__ == "__main__":
    main()

🎨 위젯 UI를 사용합니다. 스크롤 문제가 해결됩니다!

❌ 오류가 발생했습니다: 'list' object is not callable


In [2]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import threading
import shutil
from datetime import datetime
today = datetime.now().strftime("%m%d")

# Google Colab 환경 확인
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중입니다.")

def install_ytdlp():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "yt-dlp"])
    except subprocess.CalledProcessError:
        print("❌ yt-dlp 설치에 실패했습니다.")
        return False
    return True

def clear_existing_folder(folder_name):
    if os.path.exists(folder_name):
        try:
            for filename in os.listdir(folder_name):
                file_path = os.path.join(folder_name, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            print(f"🧹 기존 폴더 '{folder_name}' 내용을 모두 삭제했습니다.")
            return True
        except Exception as e:
            print(f"❌ 폴더 정리 중 오류 발생: {str(e)}")
            return False
    return True

def get_video_title(url):
    """유튜브 URL에서 영상 제목만 가져오기"""
    try:
        result = subprocess.run(
            ['yt-dlp', '--get-title', url],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return None
    except Exception:
        return None


def fetch_and_print_title(url, index):
    """스레드로 제목 가져와서 출력"""
    title = get_video_title(url)
    if title:
        print(f"\n🎵 [{index}] 제목: {title}")
    else:
        print(f"\n⚠️ [{index}] 제목 확인 실패")


def get_quality_choice():
    """화질 선택"""
    print("\n" + "="*40)
    print("🎯 화질 선택")
    print("="*40)
    print("1. 📱 일반화질 (720p 이하, 빠른 다운로드)")
    print("2. 🎬 고화질 (최고 품질, 용량 큼)")
    print("-"*40)

    while True:
        choice = input("선택하세요 (1 또는 2): ").strip()
        if choice == '1':
            print("✅ 일반화질 모드가 선택되었습니다.")
            return 'normal'
        elif choice == '2':
            print("✅ 고화질 모드가 선택되었습니다.")
            return 'high'
        else:
            print("❌ 1 또는 2를 입력해주세요!")

def create_download_folder():
    """다운로드 폴더 생성"""
    folder_name = "영상저장함"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"📁 폴더 생성: {folder_name}")
    else:
        print(f"📁 기존 폴더 사용: {folder_name}")

    return folder_name

def create_zip_file(folder_name):
    """폴더를 ZIP으로만 압축 (자동 다운로드 없음)"""
    zip_filename = f"{today}_유튜브다운로드.zip"
    if os.path.exists(zip_filename):
        os.remove(zip_filename)

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        for file in os.listdir(folder_name):
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, file)
    print(f"📦 ZIP 파일 생성 완료: {zip_filename}")
    print("💡 Colab 왼쪽 파일탐색기에서 직접 다운로드하세요.")

def get_video_urls():
    """링크만 입력 → 제목은 나중에 한번에 출력"""
    urls = []
    print("\n" + "="*80)
    print("🔗 유튜브 링크 입력")
    print("="*80)
    print("1️⃣ 다운로드할 유튜브 링크를 입력해주세요.")
    print("2️⃣ 링크를 하나씩 입력하고 엔터를 누르세요.")
    print("3️⃣ 모든 링크를 입력했으면 '다운로드'를 입력하세요.")
    print("-"*80)

    while True:
        try:
            print(f"\n🔗 링크 #{len(urls)+1} 입력:")
            url_input = input("   │ ").strip()
            if url_input.lower() == '다운로드':
                if urls:
                    print(f"\n✅ 총 {len(urls)}개 링크가 입력되었습니다.")
                    break
                else:
                    print("❌ 최소 하나의 링크를 입력해주세요!")
                    continue
            if url_input:
                if 'youtube.com' in url_input or 'youtu.be' in url_input:
                    urls.append(url_input)
                    print(f"✅ 추가됨: {len(urls)}번째 영상")
                else:
                    print("❌ 올바른 유튜브 링크를 입력해주세요!")
            else:
                print("💡 링크를 입력하거나 '다운로드'를 입력해주세요.")
        except KeyboardInterrupt:
            print("\n\n⏹️  입력이 중단되었습니다.")
            return []

    # 제목 확인은 입력이 끝난 뒤
    print("\n🎥 영상 확인 중...")
    for i, url in enumerate(urls, 1):
        title = get_video_title(url)
        if title:
            print(f"[{i}] {title}")
        else:
            print(f"[{i}] 제목 확인 실패")
    return urls


def create_zip_file(folder_name):
    zip_filename = f"{today}_유튜브다운로드.zip"
    if os.path.exists(zip_filename):
        os.remove(zip_filename)
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        for file in os.listdir(folder_name):
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, file)
    print(f"📦 ZIP 파일 생성 완료: {zip_filename}")
    return zip_filename   # ← 추가


def download_videos(urls, quality, folder_name):
    """비디오 다운로드"""
    print(f"\n🚀 다운로드 시작! (총 {len(urls)}개 영상)")
    print("="*50)

    # 화질 설정
    if quality == 'high':
        format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
        print("🎬 고화질 모드로 다운로드합니다.")
    else:
        format_option = "best[height<=720]/best"
        print("🎬 일반화질 모드로 다운로드합니다.")

    success_count = 0
    failed_urls = []

  # 수정된 다운로드 루프
    for i, url in enumerate(urls, 1):
        print(f"\n{i}/{len(urls)} 다운로드 중...")

        try:
            cmd = [
                "yt-dlp",
                "-f", format_option,
                "-o", f"{folder_name}/%(title)s.%(ext)s",
                "--no-warnings",
                url
            ]
            result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

            if result.returncode == 0:
                print(f"✅ {i}번째 영상 다운로드 완료!")
                success_count += 1
            else:
                print(f"❌ {i}번째 영상 다운로드 실패!")
                print(f"   오류 원인: {result.stderr.strip()}")
                failed_urls.append(url)

        except Exception as e:
            print(f"❌ {i}번째 영상 처리 중 예외 발생: {e}")
            failed_urls.append(url)   # ← except 안으로 넣어야 함


    print("\n" + "="*50)
    print("📊 다운로드 완료!")
    print("="*50)
    print(f"✅ 성공: {success_count}개")
    print(f"❌ 실패: {len(failed_urls)}개")

    if failed_urls:
        print("\n❌ 실패한 링크들:")
        for url in failed_urls:
            print(f"   - {url}")

    if IN_COLAB:
        print(f"\n📁 Colab 내 위치: {os.path.abspath(folder_name)}")
    else:
        print(f"\n📁 다운로드 위치: {os.path.abspath(folder_name)}")

    return success_count > 0


def remove_zip(zip_filename):
    """ZIP 파일 삭제"""
    if os.path.exists(zip_filename):
        try:
            os.remove(zip_filename)
            print(f"🗑 ZIP 파일 삭제 완료: {zip_filename}")
        except Exception as e:
            print(f"❌ ZIP 파일 삭제 실패: {e}")

def main():
    """메인 함수"""
    try:
        subprocess.run(['yt-dlp', '--version'], check=True, capture_output=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        if not install_ytdlp():
            return

    try:
        urls = get_video_urls()
        quality = get_quality_choice()
        folder_name = create_download_folder()
        has_downloads = download_videos(urls, quality, folder_name)

        if has_downloads and IN_COLAB:
          print("\n" + "="*50)
          print("📦 압축 및 다운로드 준비")
          print("="*50)
          zip_filename = create_zip_file(folder_name)
          files.download(zip_filename)
          clear_existing_folder(folder_name)
          print("⬇️ ZIP 파일 자동 다운로드 시작")

        elif has_downloads:
            print("💻 로컬 환경: 파일들이 폴더에 저장되었습니다.")

        if not IN_COLAB:
            input("\n아무 키나 눌러서 종료하세요...")

    except KeyboardInterrupt:
        print("\n\n⏹️  사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 오류가 발생했습니다: {str(e)}")
        if not IN_COLAB:
            input("\n아무 키나 눌러서 종료하세요...")

if __name__ == "__main__":
    main()



🔗 유튜브 링크 입력
1️⃣ 다운로드할 유튜브 링크를 입력해주세요.
2️⃣ 링크를 하나씩 입력하고 엔터를 누르세요.
3️⃣ 모든 링크를 입력했으면 '다운로드'를 입력하세요.
--------------------------------------------------------------------------------

🔗 링크 #1 입력:


⏹️  입력이 중단되었습니다.

🎯 화질 선택
1. 📱 일반화질 (720p 이하, 빠른 다운로드)
2. 🎬 고화질 (최고 품질, 용량 큼)
----------------------------------------


⏹️  사용자에 의해 중단되었습니다.
